In [1]:
!python -m tf2onnx.convert --tflite lightspeech_quant.tflite --output lightspeech_quant.onnx
!python -m tf2onnx.convert --tflite mbmelgan.tflite --output mbmelgan.onnx

2024-08-05 18:49:43.251011: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2024-08-05 18:49:43.271645: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 18:49:43.574192: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/s44504/miniconda3/envs/tftts2/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-08-05 18:49:44.024477: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/

In [2]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort --optimization_style=Fixed .

Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/s44504/TensorFlowTTS/notebooks/mbmelgan.onnx to ORT format model /home/s44504/TensorFlowTTS/notebooks/mbmelgan.ort
Converting optimized ONNX model /home/s44504/TensorFlowTTS/notebooks/lightspeech_quant.onnx to ORT format model /home/s44504/TensorFlowTTS/notebooks/lightspeech_quant.ort
Converted 2/2 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'
2024-08-05 18:50:20,830 ort_format_model.utils [INFO] - Created config in /home/s44504/TensorFlowTTS/notebooks/required_operators.config


In [3]:
from tensorflow_tts.inference import AutoProcessor

processor = AutoProcessor.from_pretrained("bookbot/lightspeech-mfa-sw-v2", use_auth_token=True)
processor.mode = "eval" # change processor from train to eval mode

2024-08-05 18:50:30.078759: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 18:50:30.100120: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 18:50:30.445420: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/s44504/miniconda3/envs/tftts2/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly vers

In [4]:
import onnxruntime as ort

model_path = "./lightspeech_quant.onnx"
lightspeech = ort.InferenceSession(model_path)

model_path = "./mbmelgan.onnx"
mbmelgan = ort.InferenceSession(model_path)

In [5]:
text = "Kwa maana Mungu aliupenda ulimwengu kiasi cha kumtoa Mwanae pekee, ili kila mtu amwaminiye asipotee, bali awe na uzima wa milele."
input_ids = processor.text_to_sequence(text)

In [6]:
import numpy as np

input_ids = np.array([input_ids], dtype=np.int32)
speaker_ids = np.array([0], dtype=np.int32)
speed_ratios = np.array([1.0], dtype=np.float32)
f0_ratios = np.array([1.0], dtype=np.float32)
energy_ratios = np.array([1.0], dtype=np.float32)

In [7]:
# Run inference
inputs = {
    "input_ids": input_ids,
    "speaker_ids": speaker_ids,
    "speed_ratios": speed_ratios,
    "f0_ratios": f0_ratios,
    "energy_ratios": energy_ratios,
}

mel_output, durations, _ = lightspeech.run(None, inputs)
outputs = mbmelgan.run(None, {"mels": mel_output,})
audio_tflite = outputs[0][0, :, 0]

In [8]:
import soundfile as sf
sf.write("john3.wav", audio_tflite, 44100)

In [ ]:
mel_output.shape, audio_tflite.shape

In [ ]:
durations